In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data_copy.csv')
df['director'] = df.director.str.replace(',', '')
df['critic_cn'] = df.critic_cn.str.replace('(', '')
df['critic_cn'] = df.critic_cn.str.replace(')', '')
df['movie_name'] = df.movie_name.str.replace('^\d*[.]\s', '')
# df.head()

In [ ]:
import requests
my_url = ('https://www.imdb.com/find?q=mulholland+drive')
raw_html = requests.get(my_url).content

In [ ]:
from bs4 import BeautifulSoup
soup_doc = BeautifulSoup(raw_html, 'html.parser')

In [ ]:
directors = df.director
directors_list = []
directors = set(directors)
for director in directors:
    data = {}
    string = director.replace(" ","+").lower()
    href = "https://www.imdb.com/find?q=" + string
    raw_html = requests.get(href).content
    soup_doc = BeautifulSoup(raw_html, 'html.parser')
    try:
        all_result = soup_doc.table
        td_result = all_result.find(class_="result_text")
        data['dir'] = director
        data['url'] = td_result.a["href"] 
        directors_list.append(data)
    except:
        print('No results found: '+director)
directors_list

In [ ]:
directors_birth = []
for each_dir in directors_list:
    data = {}
    try:
        director = each_dir['dir']
        href = "https://www.imdb.com/" + each_dir['url']
        raw_html = requests.get(href).content
        soup_doc = BeautifulSoup(raw_html, 'html.parser')
        all_result = soup_doc.find(id='name-born-info')
        data['director'] = each_dir['dir']
        data['href'] = "https://www.imdb.com/" + each_dir['url']
        data['director_birthYr'] = all_result.time['datetime']
        data['director_cn'] =  all_result.find_all('a')[-1].text
        directors_birth.append(data)
    except:
        print('No results found: ' +each_dir['dir'])
directors_birth                                           

In [ ]:
import pandas as pd
import numpy as np

col_names = ['director', 'href', 'director_birthYr', 'director_cn']
df2 = pd.DataFrame(directors_birth, columns=col_names)
df2.to_csv('director_data.csv', index=False)
df2.tail()

In [ ]:
df_new = pd.merge(df, df2, how='right', on='director')
df_new.to_csv('movie_data_merged.csv', index=False)

